# Model Updating

As new data comes in, it should go through the same processing pipelines as done previously (scaling, feature engineering, feature selection, dimensionality reduction, and so on). The challenge comes when how we chose to update our models.

There are several different approaches we can make use of our models. Each approach is different depending on the circumstances.


## Static Models

The procedure looks something like this:

1. Build your model with all training data.
2. Make predictions using this model

This is useful in situations where we would expect the data to change all that much. For example, hand digit recognition, or face recognition are all, for the most part, static models.

In [3]:
import numpy as np

# can alternatively use pickle, but joblib is more optimized
from sklearn.externals import joblib
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR


# suppose that our pretrained model started like this
init = 30
dataset = datasets.load_diabetes()
X, y = dataset.data, dataset.target

# data initialization
X_train = X[:-init, :]
y_train = y[:-init]

# data preprocessing - [save state]
scaler = StandardScaler()
scaler.fit(X_train)
joblib.dump(scaler, "scaler.pkl")

# fit the model with scaled data - [save state]
X_scaled_train = scaler.transform(X_train)
regressor = SVR(C=20)
regressor.fit(X_scaled_train, y_train)
joblib.dump(regressor, "regressor.pkl")

# mocking new data that comes in
X_train = X[-init:, :]
y_train = y[-init:]

# load the previous models, (if not loaded)
my_scaler = joblib.load("scaler.pkl")
my_regressor = joblib.load("regressor.pkl")

# predict
X_train = my_scaler.transform(X_train)
predictions = my_regressor.predict(X_train)
predictions

array([223.3151919 ,  94.35900362, 190.45246179, 162.88419784,
       212.92811312, 136.30183464, 117.17933268,  88.67517379,
       135.47915109, 221.68399609, 189.48544713, 163.85041462,
       148.00984505, 116.37226253, 201.66061882, 106.02869693,
       264.60286421,  93.16352761, 118.2312028 , 131.07663212,
       213.37358002,  75.78037125, 106.2207279 , 106.3541514 ,
        67.46927024, 209.90449269, 111.84487039, 101.77849948,
       190.24802448, 118.55492724])

## Small Dynamic Models

Depending on thinks like the efficiency of the training algorithm, the rate of data inflow, the size of the data, or the expected response time for a prediction, another possiblity is to completely update the model from scratch every time, or with batches of new data. This is feasable if the data is small and does not scale, or the model algorithm is efficient enough to retrain in a practical time frame.

The steps here are:

1. Train your model.
2. Make predictions.
3. Retrain your model from scratch when a new batch of information comes in.



## Online Learning/Updating

In the third case, the model is updated as batches of new information comes in, just like with _Small Dynamic Models_. The primary difference is that in this case, rather than having to completely retain from scratch, a _partial fit_ technique is employed. This option is not available to all models, (or at least not discovered yet), but when available, this option is always preferrable to _Small Dynamic Models_.

Some examples that employ _partial fit_ include:
* `IncrementalPCA`


Alternatively, numpys `memmap` classes allows manipulation of a large array stored on disk, and read it as if it were entirely stored in memory.

```python`
X_mm = np.memmap(filename, dtype="float32", mode="readonly", shape=(m, n))

batch_size = m // n_batches
inc_pca = IncrementalPCA(n_components=154, batch_size=batch_size)
inc_pca.fit(X_mm)
```

# todo ---

- Sklearn implements online learning with `partial_fit` method.
